In [6]:
# Importing necessary libraries

# Nearest neighbors will be the v1 model for sound drip
# will give logistic regressiona nd 
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.neighbors import KDTree
from joblib import dump
from joblib import load
import pickle
import numpy as np

import pandas as pd
from pandas.io.json import json_normalize

# Loading Latest PKL

In [7]:
df = pickle.load(open("./data/song_list_v6_genre_embeds.pkl", "rb"))

# Preparing for Scaling

In [8]:
df.head()

,songid,popularity,acousticness,artist,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track,valence,genres,genre_embed
0,5PS5dpaLogPzYU9hWiWyZb,0,0.001300,Karkkiautomaatti,0.487,157307.0,0.678,0.055100,9.0,0.0846,-7.780,1.0,0.0495,149.940,4.0,Tanssi vaan,0.870,"[finnish indie, suomi rock]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,41RpZW2lxAdnqDd2nMBzLQ,15,0.000045,Hudson Mohawke,0.662,138960.0,0.823,0.952000,4.0,0.3430,-1.711,0.0,0.0662,177.745,4.0,No One Could Ever,0.621,"[bass music, scottish electronic, scottish hip...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,3GsS8jzoixpCnp4jDWCEvb,17,0.002760,Kris Kross,0.859,221200.0,0.741,0.000000,11.0,0.3250,-12.329,0.0,0.2710,98.082,4.0,2 Da Beat Ch'yall,0.529,"[hip hop, new jack swing, rap]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,0kq75szR7uDEYrZkT2c4Ry,21,0.348000,Jorge Negrete,0.361,173573.0,0.483,0.000003,7.0,0.1770,-6.875,1.0,0.0287,94.538,4.0,El hijo del pueblo,0.682,"[bolero, mariachi, ranchera]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,2HyFpkX9J7vv3OZNDaraHZ,1,0.340000,Tiger Lou,0.533,318467.0,0.302,0.179000,10.0,0.1110,-10.308,1.0,0.0307,134.959,3.0,Pilots,0.294,[swedish indie rock],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [9]:
# dropping unnecessary columns
X = df.drop(["songid","artist","track", "duration_ms","genres"], axis=1)

# checking that they're dropped
X.shape

(574018, 14)

In [10]:
X

,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,genre_embed
0,0,0.001300,0.487,0.678,0.055100,9.0,0.0846,-7.780,1.0,0.0495,149.940,4.0,0.8700,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,15,0.000045,0.662,0.823,0.952000,4.0,0.3430,-1.711,0.0,0.0662,177.745,4.0,0.6210,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,17,0.002760,0.859,0.741,0.000000,11.0,0.3250,-12.329,0.0,0.2710,98.082,4.0,0.5290,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,21,0.348000,0.361,0.483,0.000003,7.0,0.1770,-6.875,1.0,0.0287,94.538,4.0,0.6820,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,1,0.340000,0.533,0.302,0.179000,10.0,0.1110,-10.308,1.0,0.0307,134.959,3.0,0.2940,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574013,33,0.798000,0.320,0.104,0.000750,0.0,0.1060,-13.162,1.0,0.0342,124.677,3.0,0.0566,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
574014,50,0.820000,0.418,0.204,0.936000,8.0,0.0866,-13.522,1.0,0.0370,110.022,4.0,0.0375,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
574015,0,0.070400,0.614,0.952,0.000000,6.0,0.3620,-2.782,1.0,0.0718,148.079,4.0,0.8200,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
574016,18,0.064000,0.343,0.487,0.000000,2.0,0.1070,-9.462,1.0,0.0473,171.217,4.0,0.3650,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


# Perform Sorting Step Later

In [13]:
X = X.reindex(sorted(X.columns), axis=1)

In [14]:
X_for_scaling = X.drop(["genre_embed"],axis=1)

In [15]:
X_genre_embeds = X["genre_embed"]

In [16]:
X_for_scaling.columns

Index(['acousticness', 'danceability', 'energy', 'instrumentalness', 'key',
       'liveness', 'loudness', 'mode', 'popularity', 'speechiness', 'tempo',
       'time_signature', 'valence'],
      dtype='object')

In [17]:
X_genre_embeds.head()

0    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: genre_embed, dtype: object

# Scaling

In [18]:
# instantiating scaler
scaler = MinMaxScaler()

# scaling data
# data_fitted = scaler.fit(X_new)
data_scaled = scaler.fit_transform(X_for_scaling)

# confirming scaling took place
data_scaled[0]

array([0.00130522, 0.4884654 , 0.678     , 0.0551    , 0.81818182,
       0.0846    , 0.78720453, 1.        , 0.        , 0.05118925,
       0.60009846, 0.8       , 0.87      ])

In [19]:
data_scaled.shape

(574018, 13)

# Normalization - Necessary Part of the Production App

In [20]:
data_scaled

array([[1.30522088e-03, 4.88465396e-01, 6.78000000e-01, ...,
        6.00098456e-01, 8.00000000e-01, 8.70000000e-01],
       [4.55823293e-05, 6.63991976e-01, 8.23000000e-01, ...,
        7.11381219e-01, 8.00000000e-01, 6.21000000e-01],
       [2.77108434e-03, 8.61584754e-01, 7.41000000e-01, ...,
        3.92549398e-01, 8.00000000e-01, 5.29000000e-01],
       ...,
       [7.06827309e-02, 6.15847543e-01, 9.52000000e-01, ...,
        5.92650255e-01, 8.00000000e-01, 8.20000000e-01],
       [6.42570281e-02, 3.44032096e-01, 4.87000000e-01, ...,
        6.85254484e-01, 8.00000000e-01, 3.65000000e-01],
       [2.93172691e-02, 3.81143430e-01, 3.99000000e-01, ...,
        5.03483965e-01, 6.00000000e-01, 1.05000000e-01]])

In [21]:
normalizer = Normalizer()
data_normalized = normalizer.fit_transform(data_scaled)

In [22]:
data_normalized.shape

(574018, 13)

# Attempting to thread massive combination of arrays

In [152]:
from multiprocessing import Pool

In [143]:
def combine_arrays():
    X_final_array = [np.array(list(data_normalized[i]) + list(X_genre_embeds[i])) for i in range(100)]
    return X_final_array

# Flattening into separate columns

In [2]:
X_final_array = [np.array(list(data_normalized[i]) + list(X_genre_embeds[i])) for i in range(len(data_normalized))]

NameError: name 'data_normalized' is not defined

# K-Nearest Neighbors Classifier w/ Data scaled and Normalized

In [171]:
# instantiating model class
knn1 = NearestNeighbors(n_neighbors=1000,algorithm='kd_tree',n_jobs=-1).fit(X_final_array)

# Attempting to Use KDTree

In [177]:
X_final_array_kd = np.array(X_final_array)

In [178]:
kdt = KDTree(X_final_array_kd, leaf_size=30, metric='euclidean')

KeyboardInterrupt: 

# Dump Model If n_neighbors param needs to be adjusted

In [172]:
dump(knn1, './data/knn_model7.joblib', compress=True)

['./data/knn_model7.joblib']

In [ ]:
dump(kdt, './data/kdt_model7.joblib', compress=True)

In [ ]:
pickle.dump(x_final_array,)

In [1]:
knn1

NameError: name 'knn1' is not defined

# Song_ID Array for Predict Flask Function

In [255]:
# # converting song id column from df to an array

# song_id_array3 = np.array(df['songid'])

# filename5 = './data/song_id_array3.pkl'
# pickle.dump(song_id_array3, open(filename5, 'wb'))

# Loading Joblibbed Model

In [ ]:
knn1 = load('./data/knn_model7.joblib')

In [4]:
# running model to test output
# knn_results1 = knn1.kneighbors([X_final_array[2]])[1]

knn1.kneighbors([X_final_array[2]])[1]

NameError: name 'X_final_array' is not defined

In [25]:
knn_results1.shape

(1, 20)

In [100]:
pd.DataFrame(song_list.loc[1005][1:11])

,1005
artist,Jean-michel Rotin
danceability,0.64
duration_ms,256133
energy,0.784
instrumentalness,0.0163
key,1
liveness,0.0355
loudness,-9.452
mode,1
songid,1uR9ohf3yyH1nygTAqqksv


In [29]:
counter = 0
for result in knn_results1[0]:
    counter += 1
    print(counter)
    display(pd.DataFrame(song_list.loc[result][10:11]),pd.DataFrame(song_list.loc[result][17:18]))

1


,1
songid,41RpZW2lxAdnqDd2nMBzLQ


,1
genres_stripped,"'bass music', 'scottish electronic', 'scottish..."


2


,7356
songid,5KrT43gJer7GAmW7lzDswI


...

,499112
songid,4u10TdQrpMjcPsYZA9tUxI


,499112
genres_stripped,"'acoustic pop', 'anthem worship', 'christian i..."


100


,497658
songid,7DLzu8CrPIl64bcbw150F1


,497658
genres_stripped,"'acoustic pop', 'anthem worship', 'christian i..."


# Model w/out Hashed Genres

In [30]:
# instantiating model class
knn2 = NearestNeighbors(n_neighbors=21,algorithm='kd_tree')

# fitting model
model2 = knn2.fit(data_scaled)

In [31]:
# running model to test output
knn_results2 = knn2.kneighbors([data_scaled[1]])[1]

In [32]:
knn_results2[0]

array([     1, 492393, 407590, 445580, 238362, 385085, 559862, 535388,
       198289,  36970, 350778,  68090, 436203,  75609,  67393, 495177,
       286524, 192022, 436810,  93001, 530514])

In [33]:
pd.DataFrame(song_list.loc[1005][1:11])

,1005
artist,Jean-michel Rotin
danceability,0.64
duration_ms,256133
energy,0.784
instrumentalness,0.0163
key,1
liveness,0.0355
loudness,-9.452
mode,1
songid,1uR9ohf3yyH1nygTAqqksv


In [34]:
counter = 0 
for result in knn_results2[0]:
    counter += 1 
    print(counter)
    display(pd.DataFrame(song_list.loc[result][10:11]),pd.DataFrame(song_list.loc[result][17:18]))

1


,1
songid,41RpZW2lxAdnqDd2nMBzLQ


,1
genres_stripped,"'bass music', 'scottish electronic', 'scottish..."


2


,492393
songid,0O3db9x93twYwR8ckduDc2


...

,93001
songid,22gpJ3mnSYHm964wFPNgeW


,93001
genres_stripped,"'ambient psychill', 'psychill'"


21


,530514
songid,6brdygAtfqp3KDceaBLDZv


,530514
genres_stripped,"'dub', 'reggae', 'roots reggae'"


In [28]:
knn_results1[0]

array([     2,  39565, 265926, 532028, 407709, 438331, 473470, 259888,
       420313, 213790, 352801, 303257, 551492, 106525, 452963,  64608,
       124604,  46919, 433612, 572160])